<a href="https://colab.research.google.com/github/Lordborack/similarcards/blob/master/ShadesV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title IMPORT
import re
import pandas as pd
import numpy as np
import nltk
import json
import math
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from operator import itemgetter 
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.metrics import f1_score
from sklearn.neighbors import NearestNeighbors
from collections import Counter

import sys
import os
import torch
import itertools

from collections import OrderedDict


pathWord2Vec = "/content/drive/My Drive/colab/resources/"
pathFastText = "/content/drive/My Drive/colab/resources/wiki-news-300d-1M.vec"



pd.set_option('precision', 2)
pd.set_option('display.max_colwidth', -1)

  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
#!pip install flask-ngrok
#!pip install -e git+https://github.com/microsoft/nlp-recipes.git@master#egg=utils_nlp
#!git clone https://github.com/microsoft/nlp-recipes.git
#!pip install gensim

from utils_nlp.models.pretrained_embeddings import word2vec
from gensim.models import KeyedVectors
from gensim.models import doc2vec
import gensim

#!pip install pytorch-pretrained-bert
#!pip install cached_property
#!pip install scrapbook

import scrapbook as sb
from utils_nlp.models.bert.common import Language, Tokenizer
from utils_nlp.models.bert.sequence_encoding import BERTSentenceEncoder, PoolingStrategy


This colab finds similarities between descriptions of sahdespire cards.


**To Do:**

* Evaluate several models
  * Create evaluation set
    * hand labeling data (100 most popular card, and find similar cards to those ones) DONE
    * load data into colab DONE 
  * create evaluation metric -> F1
    * we get the following card1,card2,N -> we output wether this card2 was whinin the N similar cards of card1.
  * increase evaluation speed  (DONE)
    * by using build in functions from NN -> this will force to set 10 similar cards per card..
    * by reducing the amount of calls to the api (50% by single call per unique card, 50% by not data duplication)
  * put config file (or chunk) -> in progress
  * create function to enrich evaluation set
  * normalize document embeding such that xT x = 1
  * implement several algorithms **(NEXT)**
    * fine tunining and reciepts from here  https://github.com/microsoft/nlp-recipes/blob/master/README.md
    * outline the algorithms that we ant to check
      * BERT_> https://github.com/microsoft/nlp-recipes/blob/master/examples/sentence_similarity/bert_encoder.ipynb
      * doc2vec -> https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e OK
      * 2 benchmarks  -> https://github.com/microsoft/nlp-recipes/blob/master/examples/sentence_similarity/baseline_deep_dive.ipynb OK
      * check entity embbeding: https://blog.insightdatascience.com/entity2vec-dad368c5b830
      * next MIN-MAX weighting (DONE)
      * self training on data doc2vec (DONE)
      * remove stop words and lematization/steaming 
      * BERT!!!!! (DONE)
  * From all recomendations of the algorithms create a set of similar cards
  * Remove the ones that we already know are similar
  * Popmpt the resutls to and end user to evaluate if they are similar or not
  * Enrich the data-set and re-evaluate the algorithms (both with the hand labeled data set and the enriched data set)

    
**Notes:**
* si una carta nomes te cartes similars que provenen de les cartes originalment similars, vol dir que el labeling es va fer bé
* si una carta nomes te cartes similars que provenen del recomanador diu que el hand labeling no es va fer bé ( i possiblement haguem de treure aqusta carata a l'hora de avaluar)
* Details on measurement metric: precision and recall?
* precision: out of the 3 similar cards all of them were similar?
  * recall: out the 3 most similar cards which were included on the set?
  * how many cards: 100-200 cards... for each we should look for similar cards: from 3 to 10. and rank them at least 3 most similar 7 lesser similar.



**Backlog:**
* remove auth when reading from GS
* clean up of data
  * improve dealing with weapons and so...
* better logic 
  * take into account resticted factions 
  * include extra features such as key words (n-gram), restrictions (to fighters wizards), ideas for evaluate: friendly and enemy should be super relevant. attack action,
  * fuzzy string matching for card names
* flask to open new tab  
* Put code into AWS and have it 24/7 (need miki for this...)
    * probably need to use AWS infrastructure
    * https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b
    * https://colab.research.google.com/github/shawwn/colab-tricks/blob/master/ngrok-tricks.ipynb#scrollTo=1B_aC562JbJU
    * https://www.twilio.com/blog/2016/08/getting-started-python-bottle-twilio-sms-mms.html














In [ ]:
#@title LOAD
def remove_white_spaces_and_special_characters(sentence):
  nowhitespaces = re.sub('[^A-Za-z0-9 ]+', '', sentence)
  return(nowhitespaces)

def clean_description(text):
    text = re.sub('[Mm]ag[Cc]hannel', 'Magical Channel', text)
    text = re.sub('\:dfDodge\:','Dodge', text)
    text = re.sub('[Ee]rrata update', '', text)
    text = re.sub('-','minus ', text)
    text = re.sub('\*','',text)
    text = re.sub('\\[tnr]','',text)
    text = re.sub('[^A-Za-z0-9 ]+', '', text)

    return(text)

def clean_cards(data):
  data['Description_clean'] = data['Description_clean'].apply(str)
  data['Name'] = data['Name'].apply(str)
  data = data[['Number','Name','Type','Faction','Description_clean']]
  data = data.drop_duplicates(subset = 'Name')
  data['Description_clean'] = [clean_description(name).lower() for name in data['Description_clean']]

  data = data[data['Description_clean'] != 'nan']
  data = data[data['Name'] != 'nan']
  data['Name'] = [name.lower() for name in data['Name']]
  data = data.dropna(0)
  return(data)

def read_from_gs(path, sheet_name):
  from google.colab import auth
  auth.authenticate_user()

  import gspread
  from oauth2client.client import GoogleCredentials

  gc = gspread.authorize(GoogleCredentials.get_application_default())
  wb = gc.open_by_url(path)
  sheet = wb.worksheet(sheet_name)

  records = sheet.get_all_values()
  data = pd.DataFrame.from_records(records[1:])
  dic_rename = dict(zip(data.columns.tolist(), records[0]))
  data.rename(columns = dic_rename, inplace = True)
  
  return(data)

def read_and_clean_gs(path, sheet_name):
  
  data = read_from_gs(path, sheet_name)
  data.rename(columns = {'Description':'Description_clean'}, inplace = True)
  return(clean_cards(data))


def read_and_clean_csv(path):
  df = pd.read_csv(path)
  df.rename(columns= {'Description.1':'Description_clean'}, inplace = True)
  return(clean_cards(df))

def get_id_card_name_dic(cards):
  dic = dict(zip(cards['Number'], cards['Name']))
  return(dic)  

def get_card_name_id_dic(cards):
  dic = dict(zip(cards['Name'], cards['Number']))
  return(dic)

def read_and_clean_validation_set(path, sheet_name):
  df = read_from_gs(path, sheet_name)
  data = pd.melt(df, id_vars=["Name"])
  data = data[data['variable'] != 'Description']
  data = data[['Name', 'value']]
  data = data[data['value'] != '']

  data = data.dropna(0)
  data[data['Name'] == 'Combination Strike']
  data['Name'] = [name.lower() for name in data['Name']]
  data['value'] = [name.lower() for name in data['value']]

  data2 = data.copy()
  data2.rename(columns = {'Name':'value', 'value':'Name'}, inplace = True)
  data2.shape

  data3 = pd.concat([data2, data])
  data3 = data3.drop_duplicates()


  data_num_by_name = data3.groupby('Name').count()
  data_num_by_name.rename(columns = {'value':'num'}, inplace = True)
  data4 = pd.merge(data3, data_num_by_name, on = "Name")
  data4 = data4[['Name','value','num']]

  return(data4)

In [ ]:
#@title EMBEDDINGS
CLOSER_POINTS_ALG = "KNN"
EMBEDDINGDS = "GLOVE_MIN_MAX"
N_NEIGHBORS = 10
 
def setup_bert():
  # device config
  NUM_GPUS = 0

  # model config
  LANGUAGE = Language.ENGLISH
  TO_LOWER = True
  MAX_SEQ_LENGTH = 128
  LAYER_INDEX = -2
  POOLING_STRATEGY = PoolingStrategy.MEAN

  # path config
  CACHE_DIR = "./temp"

  if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR, exist_ok=True)

  bert_se = BERTSentenceEncoder(
    language=LANGUAGE,
    num_gpus=NUM_GPUS,
    cache_dir=CACHE_DIR,
    to_lower=TO_LOWER,
    max_len=MAX_SEQ_LENGTH,
    layer_index=LAYER_INDEX,
    pooling_strategy=POOLING_STRATEGY,
  )
  return(se)


def train_doc2vec(cards):
  cards_clean = [tokenizize_and_clean(s) for s in cards['Description_clean'] ]
  train_corpus = list()
  i = -1
  for tokens in cards_clean:
    i = i + 1
    train_corpus.append(gensim.models.doc2vec.TaggedDocument(tokens, [i]))

  model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40, dm = 0)
  model.build_vocab(train_corpus)
  model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
  return(model)

def get_embeddings_glove():
  print('Indexing word vectors.')
  embeddings_index = {}
  f = open('/content/drive/My Drive/colab/resources/glove.6B.'+str(embedding_dim)+'d.txt', encoding='utf-8')
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print('Found %s word vectors.' % len(embeddings_index))
  return(embeddings_index)

def get_document_frequency(cards):
    document_frequency_dict = {}
    all_sentences = [tokenizize_and_clean(s) for s in cards['Description_clean'] ]
    sentence = np.concatenate(all_sentences, axis=0)
    n = len(sentence)

    for s in all_sentences:
        for token in set(s):
            document_frequency_dict[token] = document_frequency_dict.get(token, 0) + 1

    return document_frequency_dict, n
  
def doc_embeddings_weighted_average(doc, embeddings_index):
    # Throw away tokens that are not in the embedding model
    tokens = [i for i in doc if i in embeddings_index]
    if len(tokens) == 0:
        return []

    # We will weight by TF-IDF. The TF part is calculated by:
    # (# of times term appears / total terms in sentence)
    count = Counter(tokens)
    token_list = list(count)
    term_frequency = [count[i] / len(tokens) for i in token_list]

    # Now for the IDF part: LOG(# documents / # documents with term in it)
    inv_doc_frequency = [
        math.log(num_documents / (document_frequencies.get(i, 0) + 1)) for i in count
    ]

    # Put the TF-IDF together and produce the weighted average of vector embeddings
    word_embeddings = [embeddings_index[token] for token in token_list]
    weights = [term_frequency[i] * inv_doc_frequency[i] for i in range(len(token_list))]
    doc_embeddings = np.average(word_embeddings, weights=weights, axis=0)
    return normalize(doc_embeddings)

def normalize(x):
  return(x / np.sqrt(np.sum(x**2)))

def doc_embeddings_min_max(doc, embeddings_index):
  embedding_dim = len(embeddings_index['1'])

  w_max_embeddings = -float("inf")*np.ones((1,embedding_dim))
  w_min_embeddings = float("inf")*np.ones((1,embedding_dim))  
  tokens = [i for i in doc if i in embeddings_index]

  for word in tokens:
    w_embedding = embeddings_index[word]
    w_max_embeddings = np.maximum(w_embedding.astype(float), w_max_embeddings)
    w_min_embeddings = np.minimum(w_embedding.astype(float), w_max_embeddings)

  w_min_max_embeddings = np.concatenate([w_min_embeddings, w_max_embeddings], axis = 1).flatten()
  return normalize(w_min_max_embeddings)

def tokenizize_and_clean(sentence):
  stop_words = set(stopwords.words('english')) 
  word_tokens = word_tokenize(sentence)
  filtered_sentence = [w.lower() for w in word_tokens if (not w in stop_words) and w.isalpha()] 
  return filtered_sentence

def glove_min_max_sentence_embeddings(cards):
  cards_clean = [tokenizize_and_clean(s) for s in cards['Description_clean'] ]
  cards_clean_embedding = [doc_embeddings_min_max(s, glove_embeddings) for s in cards_clean]
  return (cards_clean_embedding)

def word2vec_ifdf_sentence_embeddings(cards):
   cards_clean = [tokenizize_and_clean(s) for s in cards['Description_clean'] ]
   cards_clean_embedding = [doc_embeddings_weighted_average(s, word2vec_embeddings) for s in cards_clean]
   return (cards_clean_embedding)

def word2vec_min_max_sentence_embeddings(cards):
   cards_clean = [tokenizize_and_clean(s) for s in cards['Description_clean'] ]
   cards_clean_embedding = [doc_embeddings_min_max(s, word2vec_embeddings) for s in cards_clean]
   return (cards_clean_embedding)

def glove_ifdf_sentence_embeddings(cards): 
  cards_clean = [tokenizize_and_clean(s) for s in cards['Description_clean'] ]
  cards_clean_embedding = [doc_embeddings_weighted_average(s, glove_embeddings) for s in cards_clean]
  return (cards_clean_embedding)

def fastext_ifdf_sentence_embeddings(cards): 
  cards_clean = [tokenizize_and_clean(s) for s in cards['Description_clean'] ]
  cards_clean_embedding = [doc_embeddings_weighted_average(s, fastext_embeddings) for s in cards_clean]
  return (cards_clean_embedding)

def fastext_min_max_sentence_embeddings(cards): 
  cards_clean = [tokenizize_and_clean(s) for s in cards['Description_clean'] ]
  cards_clean_embedding = [doc_embeddings_min_max(s, fastext_embeddings) for s in cards_clean]
  return (cards_clean_embedding)

def doc2vec_embeddings(cards):
  cards_clean = [tokenizize_and_clean(s) for s in cards['Description_clean'] ]
  cards_clean_embedding = [doc2vecSelfTrainedModel.infer_vector(s) for s in cards_clean]
  return (cards_clean_embedding)

def bert_embeddings(cards):
  cards_clean = [tokenizize_and_clean(s) for s in cards['Description_clean'] ]
  cards_clean_single_string = [" ".join(s) for s in cards_clean ]
  cards_clean_embedding = bert_se.encode(cards_clean_single_string, as_numpy = True)
  return (cards_clean_embedding)



embeddings_dispatcher = {   "GLOVE_MIN_MAX" : glove_min_max_sentence_embeddings, 
                            "GLOVE_WEIGTHED_AVERAGE" : glove_ifdf_sentence_embeddings,
                            "WORD2VEC_WEIGTHED_AVERAGE": word2vec_ifdf_sentence_embeddings,
                            "WORD2VEC_MIN_MAX":word2vec_min_max_sentence_embeddings, 
                            "FASTTEXT_WEIGTHED_AVERAGE":fastext_ifdf_sentence_embeddings,
                            "FASTTEXT_MIN_MAX":fastext_min_max_sentence_embeddings,
                            "DOC2VEC_SELFTRAINED": doc2vec_embeddings,
                            "BERT": bert_embeddings}

                            
def generate_card_embeddings_deck(cards):

  emb = embeddings_dispatcher[EMBEDDINGDS]
  cards_embedding = emb(cards)
  deck = {}
  i = 0
  for number in cards['Number']: 
    deck[number] = cards_embedding[i]
    i=i+1
  return(deck)

def knn_model():
    nbrs = NearestNeighbors(n_neighbors = N_NEIGHBORS + 1, algorithm='ball_tree').fit(np.array(list(deck.values())))
    return(nbrs)

def knn_similarity(id):
    similarity={} 
    key_idx = np.array(list(deck.keys()))  
    distance, index = nbrsModel.kneighbors(deck[id].reshape(1, -1))
    similarids = key_idx[index].flatten()
    distance = distance.flatten()

    i = 0
    for key in similarids[similarids!=id]:
      similarity[key]= 1/(1 + distance[i])
      i = i +1

    return(similarity)

def brute_force_cosine_similarity(id):
    deckSimilarity={}
    cardEmbedding = deck.get(id)
    keys = np.array(list(deck.keys()))
    keys = keys[keys!=id]

    for key in keys:
      cardEmbedding2 = deck.get(key)
      similarity = cosine_similarity(cardEmbedding.reshape(1, -1), cardEmbedding2.reshape(1, -1))
      deckSimilarity[key] = similarity.item()

    res = dict(sorted(deckSimilarity.items(), key = itemgetter(1), reverse = True)[:N_NEIGHBORS]) 
    return(res)

closer_distance_dispatcher = { "KNN": knn_similarity, "BFCS": brute_force_cosine_similarity }



In [ ]:
#@title EVAL & DECORATORS
def get_card_faction(number, cards):
    return cards.loc[list(cards['Number'] == number).index(True),'Faction']   

def remove_factions_not_universal_and_not_equal_to(faction, cards):
    cardsToKeep = cards[cards['Faction'].isin([faction,'Universal'])]['Number']
    return { key: deck[key] for key in cardsToKeep }

def get_card_number(card_name):
  for key, value in id_card_name.items():
    if str(value) == card_name:
      return(key)

def deck_decorator(card_number, cards):
    return(cards[ cards['Number'].isin(card_number)])

def get_similar_card(cardName):
    id = get_card_number(cardName.lower())
    cardEmbedding = deck.get(id)
    findCloserCards = closer_distance_dispatcher[CLOSER_POINTS_ALG]

    if cardEmbedding is not None:
      res = findCloserCards(id)
    else :
      res = 'Card ' + cardName + ' Not Found '
      #remove_factions_not_universal_and_not_equal_to(get_card_faction(number, cards), cards)
    return(res)
    
def predict(validation_df):

  num_rows = validation_df.shape[0]
  y = np.zeros((num_rows,1))
  y_true = np.ones((num_rows, 1))
  for index, row in validation_df.iterrows():
    similar_cards = get_similar_card(row['Name'])
    similar_cards_id = get_similar_card(row['Name'])
    similar_cards_name = [id_card_name[key] for key in similar_cards.keys()]
    if row['value'] in similar_cards_name:
      similar = 1
    else: 
      similar = 0
    y[index] = similar

  return(y)

In [ ]:
#cards = read_and_clean_csv('/content/drive/My Drive/colab/resources/shadespiredata.csv')
cards = read_and_clean_gs('https://docs.google.com/spreadsheets/d/1FLfMz6FI3EyhxUwS9piCqO-IqiyGOhezQrVe3dW4dp8/edit#gid=2127126048', 'DataClean')
validation_df = read_and_clean_validation_set('https://docs.google.com/spreadsheets/d/1FLfMz6FI3EyhxUwS9piCqO-IqiyGOhezQrVe3dW4dp8/edit#gid=2127126048','100CardsAndItsSimilar' )
id_card_name = get_id_card_name_dic(cards)
card_name_id = get_card_name_id_dic(cards)

In [ ]:
glove_embeddings = get_embeddings_glove()
word2vec_embeddings = word2vec.load_pretrained_vectors(dir_path=pathWord2Vec)
fastext_embeddings = KeyedVectors.load_word2vec_format(pathFastText)
doc2vecSelfTrainedModel = train_doc2vec(cards)
bert_se = setup_bert()
document_frequencies, num_documents = get_document_frequency(cards)

#name = '/content/drive/My Drive/colab/resources/glove.6B.matrix'+str(embedding_dim)+'.json'
#a_file = open(name, "w")
#json.dump(glove_embeddings, a_file)
#a_file.close()

Indexing word vectors.
Found 400000 word vectors.
Vector file already exists. No changes made.


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
#regenerate for each embbeding and model
EMBEDDINGDS = "GLOVE_MIN_MAX"
deck = generate_card_embeddings_deck(cards)
nbrsModel = knn_model()
y_predict = predict(validation_df)
y_true = np.ones((validation_df.shape[0],1))
f1_score(y_true, y_predict)

0.48577680525164113

In [ ]:
#regenerate for each embbeding and model
EMBEDDINGDS = "GLOVE_WEIGTHED_AVERAGE"
deck = generate_card_embeddings_deck(cards)
nbrsModel = knn_model()
y_predict = predict(validation_df)
y_true = np.ones((validation_df.shape[0],1))
f1_score(y_true, y_predict)

0.5118279569892473

In [ ]:
EMBEDDINGDS = "WORD2VEC_WEIGTHED_AVERAGE"
deck = generate_card_embeddings_deck(cards)
nbrsModel = knn_model()
y_predict = predict(validation_df)
y_true = np.ones((validation_df.shape[0],1))
f1_score(y_true, y_predict)

0.5717234262125902

In [ ]:
EMBEDDINGDS = "WORD2VEC_MIN_MAX"
deck = generate_card_embeddings_deck(cards)
nbrsModel = knn_model()
y_predict = predict(validation_df)
y_true = np.ones((validation_df.shape[0],1))
f1_score(y_true, y_predict)

0.460511679644049

In [ ]:
EMBEDDINGDS = "FASTTEXT_WEIGTHED_AVERAGE"
deck = generate_card_embeddings_deck(cards)
nbrsModel = knn_model()
y_predict = predict(validation_df)
y_true = np.ones((validation_df.shape[0],1))
f1_score(y_true, y_predict)

0.5354497354497355

In [ ]:
EMBEDDINGDS = "FASTTEXT_MIN_MAX"
deck = generate_card_embeddings_deck(cards)
nbrsModel = knn_model()
y_predict = predict(validation_df)
y_true = np.ones((validation_df.shape[0],1))
f1_score(y_true, y_predict)

0.4890829694323144

In [ ]:
EMBEDDINGDS = "DOC2VEC_SELFTRAINED"
deck = generate_card_embeddings_deck(cards)
nbrsModel = knn_model()
y_predict = predict(validation_df)
y_true = np.ones((validation_df.shape[0],1))
f1_score(y_true, y_predict)

0.4236902050113895

In [ ]:
EMBEDDINGDS = "BERT"
deck = generate_card_embeddings_deck(cards)
nbrsModel = knn_model()
y_predict = predict(validation_df)
y_true = np.ones((validation_df.shape[0],1))
f1_score(y_true, y_predict)

0.4570791527313266

AttributeError: ignored

  Created wheel for scrapbook: filename=scrapbook-0.2.0-cp36-none-any.whl size=4883 sha256=01eae969b634a695b521132424977263f9c3caeb6c628b794c52194ddf7be812
  Stored in directory: /root/.cache/pip/wheels/35/4e/3e/b23f6c97e73fdcae44d7a5c8c22024b8e9410c1a20fe61f864
Successfully built scrapbook


In [ ]:




emb = bert_embeddings(cards)


100%|██████████| 1423/1423 [00:00<00:00, 3969.28it/s]


In [ ]:
bert_embedding = emb

In [ ]:
cards_embedding = bert_embedding
deck = {}
i = 0
for number in cards['Number']: 
  deck[number] = cards_embedding[i]
  i=i+1

In [ ]:
similar_cards = get_similar_card('Pit trap')
similar_cards_decorated = deck_decorator(similar_cards.keys(), cards)
similar_cards_decorated

,Number,Name,Type,Faction,Description_clean
29,30,awe-inspiring,Objective,Steelheart's Champions,score this immediately if your warband has taken two or more fighters out of action in this phase able to be scored if you draw this card after the condition was met
355,356,shardgale,Ploy,Universal,all fighters suffer 1 damage
540,N44,rending scream,Spell,Thorns of the Briar Queen,gambit spell magical channelmagical channel if this spell is cast each enemy fighter adjacent to the caster suffers 1 damage
591,N95,scragged,Objective,Zarbag's Gitz,score this immediately if an enemy fighter is taken out of action while adjacent to three or more friendly fighters
736,N240,sound finances,Objective,Thundrik's Profiteers,score this in an end phase if you have 5 or more unspent glory points
914,N418,grievous riposte,Ploy,Universal,reaction play this after an attack action with a range of 1 that targets a friendly fighter and fails if there are any atcrit in your defence roll the attacker suffers 2 damage
975,N479,blessing of argentine,Upgrade,Universal,reaction during an adjacent enemys attack action that damages this fighter their attacker suffers 1 damage
991,N495,enchanted greaves,Upgrade,Universal,this fighter treats lethal hexes as normal hexes
1591,B354,lucky escape,Ploy,Universal,reaction play this when a friendly fighter is dealt 1 damage precisely that damage is not dealt to them
1605,B368,snare,Ploy,Universal,reaction play this during a friendly hunters attack action after it drives back an enemy fighter the enemy fighter is dealt 1 damage



# This is the code for using flask and pigeon
 



In [ ]:
def similarcards(name):
    similar_cards = get_similar_card(name, 3, cards)
    if  isinstance(similar_cards, str):
      cardsToReturn = similar_cards
    else:
      similar_cards_decorated = deck_decorator(similar_cards.keys(), cards).copy()
      similar_cards_decorated['Score'] = similar_cards.values()
      similar_cards_decorated.sort_values(by=['Score'])
      cardsToReturn = similar_cards_decorated.to_dict('index')
    return cardsToReturn

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, redirect
app = Flask(__name__)
run_with_ngrok(app) 
@app.route('/similarcards/<name>')

#@app.route('/similarcards/<name>')


def similarcardsUWDB(name):
    similar_cards = get_similar_card(name, 6, cards)
    if isinstance(similar_cards, str):
      cardsToReturn = similar_cards
    else:  
      base_url='https://www.underworldsdb.com/shared.php?deck=0,'
      similarCards_url = ','.join([str(elem) for elem in similar_cards.keys()])
      url = base_url + similarCards_url
      cardsToReturn = redirect(url, code=302)
    return cardsToReturn

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://dd71460272d3.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Jul/2020 12:25:29] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/Jul/2020 12:25:35] "GET /simia HTTP/1.1" 404 -
127.0.0.1 - - [09/Jul/2020 12:25:36] "GET /simi HTTP/1.1" 404 -
127.0.0.1 - - [09/Jul/2020 12:25:39] "GET /similarcard/ HTTP/1.1" 404 -
127.0.0.1 - - [09/Jul/2020 12:25:42] "GET /similarcard/distraction HTTP/1.1" 404 -
127.0.0.1 - - [09/Jul/2020 12:26:59] "GET /similarcards/distraction HTTP/1.1" 302 -
127.0.0.1 - - [09/Jul/2020 12:27:21] "GET /similarcards/distraction HTTP/1.1" 302 -
127.0.0.1 - - [09/Jul/2020 12:27:22] "GET /similarcards/distraction HTTP/1.1" 302 -
127.0.0.1 - - [09/Jul/2020 12:27:24] "GET /similarcards/d HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2020 12:27:25] "GET /similarcards/ HTTP/1.1" 404 -
127.0.0.1 - - [09/Jul/2020 12:27:31] "GET /similarcards/si HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2020 12:27:31] "GET /similarcards/sid HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2020 12:27:31] "GET /similarcards/side HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2020 12

In [ ]:
from pigeon import annotate
annotations = annotate(
  ['I love this movie', 'I was really disappointed by the book'],
  options=['positive', 'negative']
)

HTML(value='0 examples annotated, 3 examples left')

Output()

Annotation done.


[('I love this movie', 'positive'),
 ('I was really disappointed by the book', 'negative')]

In [ ]:
!pip install pigeon-jupyter

  Created wheel for pigeon-jupyter: filename=pigeon_jupyter-0.1.0-cp36-none-any.whl size=3866 sha256=56d0b611d69a5f59f16460156ceddc5c0dbdbd56df59a56dd87989e5982303fe
  Stored in directory: /root/.cache/pip/wheels/9c/e1/74/712f424efed3467a38edd654aedaad8e303e254a5a0fdbed49
Successfully built pigeon-jupyter
